# The Scoring Logic

# Setup

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys
from dotenv import load_dotenv
from supabase import create_client
import warnings
warnings.filterwarnings("ignore")

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from utils import get_table_from_supabase, build_relationship_cols, build_financial_history

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

----

# Retrieving Data from Supabase and Building Dataframes

As with my EDA, I will connect to Supabase and retrieve all records, I will create one dataframe for funder information, and another for grants and recipients information. This will allow me to easily access funders' giving history, plus the classifications for both funders and recipients, to be used as part of the calculation of the alignment score.

In [2]:
#get tables and build dataframes
tables = ["funders", "causes", "areas", "beneficiaries", "grants",
               "funder_causes", "funder_areas", "funder_beneficiaries", "funder_grants", 
               "financials", "funder_financials",
               "embedding_pairs", "evaluation_pairs", "logic_pairs",
               "area_hierarchy"]

for table in tables:
    globals()[table] = get_table_from_supabase(url, key, table)

#get recipients with filter
recipients = get_table_from_supabase(url, key, "recipients", batch_size=50, filter_recipients=True)
all_recipient_ids = set(recipients["recipient_id"].unique())

#get and filter recipient join tables
recipient_join_tables = ["recipient_grants", "recipient_areas", "recipient_beneficiaries", "recipient_causes"]
for table in recipient_join_tables:
    df = get_table_from_supabase(url, key, table)
    globals()[table] = df[df["recipient_id"].isin(all_recipient_ids)]

## The Funders Dataframe

### Main Table

In [3]:
funders_df = funders.copy()

#define table relationships for funders
funder_rels = [
    {
        "join_table": funder_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "causes"
    },
    {
        "join_table": funder_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "areas"
    },
    {
        "join_table": funder_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "beneficiaries"
    }
]

#add relationship columns
funders_df = build_relationship_cols(funders_df, "registered_num", funder_rels)

#round to 2 decimal places
funders_df = funders_df.round(2)
pd.set_option("display.float_format", "{:.2f}".format)

### Financial History Table

In [4]:
funders_df = build_financial_history(funders_df, "registered_num", funder_financials, financials)

### The List Entries

In [5]:
#get list entries
list_entries = get_table_from_supabase(url, key, "list_entries")
funder_list = get_table_from_supabase(url, key, "funder_list")
list_with_info = funder_list.merge(list_entries, on="list_id")

#get list of entries for each funder
list_grouped = list_with_info.groupby("registered_num")["list_info"].apply(list).reset_index()
list_grouped.columns = ["registered_num", "list_entries"]

#merge with funders and replace nans
funders_df = funders_df.merge(list_grouped, on="registered_num", how="left")
funders_df["list_entries"] = funders_df["list_entries"].apply(lambda x: x if isinstance(x, list) else [])

In [5]:
#extend column view, sort and preview funders
pd.set_option("display.max_columns", 100)
funders_df = funders_df.sort_values("name")
funders_df.head()

,registered_num,name,website,activities,objectives,income_latest,expenditure_latest,objectives_activities,achievements_performance,grant_policy,is_potential_sbf,is_on_list,is_nua,name_em,activities_em,objectives_em,objectives_activities_em,achievements_performance_em,grant_policy_em,concat_em,causes,areas,beneficiaries,income_history,expenditure_history,list_entries
595,1199830,1887 ALFRED SHARP BINGLEY EDUCATIONAL TRUST,None,TO ADVANCE THE EDUCATION OF CHILDREN AND ADULT...,TO ADVANCE THE EDUCATION OF CHILDREN AND ADULT...,11764.0,85104.0,None,None,None,False,False,False,"[-0.034516126,0.03585861,-0.019434275,-0.00770...","[-0.0012041864,0.038664076,-0.021148162,-0.044...","[-0.026964815,0.03308201,-0.013108682,-0.02585...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.021101058,-0.005593046,0.018036723,-0.0653...",[Education/training],"[Bradford City, West Yorkshire]",[Other Charities Or Voluntary Bodies],"{2023: 5392.0, 2024: 11764.0}","{2023: 0.0, 2024: 85104.0}",[]
468,1178603,21SIX FOUNDATION,None,TO ADVANCE SUCH CHARITABLE PURPOSES (ACCORDING...,1) TO ADVANCE SUCH CHARITABLE PURPOSES (ACCORD...,0.0,0.0,None,None,None,False,False,False,"[0.006271659,0.004286385,-0.0053785853,-0.0276...","[-0.021429172,0.029441489,-0.0057669384,-0.029...","[-0.007718874,0.029303301,-0.020673396,-0.0486...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.0028990973,0.029799648,0.0061112936,-0.032...","[General Charitable Purposes, Education/traini...",[Throughout England And Wales],"[Children/young People, Other Charities Or Vol...","{2020: 0.0, 2021: 0.0, 2022: 0.0, 2023: 0.0, 2...","{2020: 0.0, 2021: 0.0, 2022: 0.0, 2023: 0.0, 2...",[]
638,200198,29TH MAY 1961 CHARITY,https://www.29may1961charity.org.uk,THE CHARITY IS A GENERAL GRANT MAKING TRUST,SUCH CHARITABLE PURPOSES AS THE TRUSTEES IN TH...,6307930.0,6307930.0,THE 29TH MAY 1961 CHARITABLE TRUST IS A GENERA...,THE TRUSTEES' MAIN AIMS ARE THREEFOLD: TO CONT...,THE POLICY OF THE TRUSTEES IS TO GIVE CAREFUL ...,False,False,False,"[-0.0023942178,0.0036141651,0.0071925735,-0.03...","[-0.01521982,0.04057352,-0.021413913,-0.008342...","[0.019806694,0.002071932,-0.0149501925,-0.0381...","[-0.0054083704,0.0076256897,0.0032458683,-0.03...","[-0.003699224,0.014419855,-0.014209436,-0.0185...","[-0.009677714,0.017033687,-0.024075136,-0.0202...","[-0.013430712,-0.00056217087,-0.013404356,-0.0...",[General Charitable Purposes],"[Scotland, Europe, Northern Ireland, Throughou...",[Other Charities Or Voluntary Bodies],"{2020: 5382630.0, 2021: 5653027.0, 2022: 61524...","{2020: 5468431.0, 2021: 5685885.0, 2022: 61524...",[]
193,1109733,3 TS CHARITABLE TRUST,None,GENERAL CHARITABLE PURPOSES BOTH NATIONALLY AN...,1. FOR SUCH CHARITABLE PURPOSES AS THE TRUSTE...,973236.0,321876.0,None,None,None,False,False,True,"[0.04665087,-0.009608634,-0.018157814,-0.03063...","[-0.016648166,0.0056061265,-0.001358426,-0.012...","[0.021857645,0.03660929,-0.023904005,-0.013620...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.002840451,0.021916328,0.010091798,-0.04986...",[General Charitable Purposes],"[India, Asia, Throughout England And Wales]",[The General Public/mankind],"{2020: 518307.0, 2021: 1585536.0, 2022: 165729...","{2020: 592345.0, 2021: 1639987.0, 2022: 175750...",[]
596,1199978,360 GRASS ROOTS FOUNDATION,None,TO RELIEVE THE NEEDS OF SOCIALLY AND ECONOMICA...,"FOR THE PUBLIC BENEFIT, TO RELIEVE THE NEEDS O...",6540.0,3021.0,None,None,None,False,False,False,"[0.013442943,-0.023781048,-0.00063610706,0.015...","[-0.033052746,0.0033413314,-0.0021838336,-0.06...","[-0.000885371,0.002130016,-0.006337011,-0.0433...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.0198177

In [7]:
#get checkpoint folder
checkpoint_folder = Path("./10.1_checkpoints/")

#create checkpoint - save df to pickle
# funders_df.to_pickle(checkpoint_folder / "funders_df.pkl")
# print("Saved funders_df to checkpoint")

Saved funders_df to checkpoint


## The Grants Dataframe

### Main Table

In [8]:
grants_df = grants.copy()

#ddd funder info
grants_df = grants_df.merge(funder_grants, on="grant_id")
grants_df = grants_df.merge(funders[["registered_num", "name"]], on="registered_num")
grants_df = grants_df.rename(columns={"name": "funder_name", "registered_num": "funder_num"})

#ddd recipient info  
grants_df = grants_df.merge(recipient_grants, on="grant_id")
grants_df = grants_df.merge(recipients[["recipient_id", "recipient_name", "recipient_activities", "recipient_objectives", 
                                        "recipient_name_em", "recipient_activities_em", "recipient_objectives_em", "recipient_concat_em", "is_recipient"]], 
                        on="recipient_id", 
                        how="left")

#define relationships for recipients
recipient_rels = [
    {
        "join_table": recipient_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "recipient_areas"
    },
    {
        "join_table": recipient_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "recipient_causes"
    },
    {
        "join_table": recipient_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "recipient_beneficiaries"
    }
]

#add relationship columns
grants_df = build_relationship_cols(grants_df, "recipient_id", recipient_rels)

#add source of grant
grants_df["source"] = grants_df["grant_id"].apply(lambda x: "Accounts" if str(x).startswith("2") else "360Giving")

#round to 2 decimal places
grants_df = grants_df.round(2)

In [6]:
#sort and preview grants
grants_df = grants_df.sort_values("grant_title", ascending=True)
grants_df.head()

,grant_title,grant_desc,amount,year,grant_id,source,grant_title_em,grant_desc_em,grant_concat_em,funder_num,funder_grants_id,funder_name,recipient_id,recipient_grants_id,recipient_name,recipient_activities,recipient_objectives,recipient_name_em,recipient_activities_em,recipient_objectives_em,recipient_concat_em,is_recipient,recipient_areas,recipient_causes,recipient_beneficiaries
24592,"""A BLUE NEW DEAL FOR UK COSTAL COMMUNITIES"" PR...","TOWARDS THE COSTS OF THE 'BLUE NEW DEAL', WHIC...",90000.0,2014,360G-Ellerman-15189,360Giving,"[-0.02747339,0.033531465,0.009024051,-0.019257...","[-0.009053092,0.048324082,0.010487654,-0.00654...","[-0.04079483,0.026323726,0.04434301,-0.0098600...",263207,44987,JOHN ELLERMAN FOUNDATION,1055254,43799,NEW ECONOMICS FOUNDATION,NEF AIMS TO MAXIMISE WELL-BEING AND SOCIAL JUS...,A) TO ADVANCE EDUCATION AND UNDERTAKE RESEARCH...,"[0.03333197,0.033027016,-0.021830654,0.0019864...","[-0.0035722542,-0.009917363,-0.05721067,-0.024...","[0.051222906,-0.0043176734,-0.015792407,-0.001...","[0.0069951112,-0.0064328616,-0.022587229,-0.02...",True,[Throughout England And Wales],"[Education/training, Economic/community Develo...",[The General Public/mankind]
29639,"""BEFRIENDING COFFEE CLUB""",THE BEFRIENDING COFFEE CLUB WAS A TWO-YEAR PRO...,27313.0,2017,360G-PeoplesHealthTrust-2017_5,360Giving,"[-0.012254217,0.05368498,-0.036267474,-0.03152...","[0.019002143,-0.011056782,-0.035396896,-0.0086...","[0.014113758,-0.00547095,-0.01953808,-0.011693...",1125537,41014,PEOPLE'S HEALTH TRUST,1166949,39824,SANDWELL AFRICAN WOMEN ASSOCIATION,1. ADVICE & SUPPORT\rA) GENERAL HELP WITH CLAI...,THE PURPOSE OF SANDWELL AFRICAN WOMEN ASSOCIAT...,"[0.0030897702,-0.060182624,-0.029301269,-0.035...","[0.014327871,-0.03869127,-0.028160453,-0.00511...","[-0.029044177,-0.024256356,-0.02103953,-0.0508...","[-0.0010344738,-0.019625172,-0.012343395,-0.04...",True,"[Sandwell, West Midlands]","[General Charitable Purposes, The Advancement ...","[Children/young People, Elderly/old People, Pe..."
29462,"""BETTER LIVES""",THIS TWO-YEAR PROJECT DELIVERED ACTIVITIES AND...,43982.0,2017,360G-PeoplesHealthTrust-2017_103,360Giving,"[0.022505702,0.043717444,-0.022319954,0.005562...","[-0.011303271,0.020975992,-0.050781097,-0.0160...","[0.00070294796,0.013241124,-0.046168294,0.0105...",1125537,40917,PEOPLE'S HEALTH TRUST,1093240,39727,SOUTHEND CARERS,"TO PROVIDE INFORMATION, ADVICE AND SUPPORT TO ...","TO RELIEVE MENTALLY OR PHYSICALLY ILL PERSONS,...","[0.003565758,0.02407322,-0.03889863,-0.0255322...","[-0.0056407717,0.020441514,-0.03683357,-0.0246...","[-0.06805133,0.04419574,-0.019360168,-0.021728...","[-0.025895609,-0.011933744,-0.031101825,-0.008...",True,"[Southend-on-sea, Essex]","[Disability, Other Charitable Purposes]","[Children/young People, Elderly/old People, Pe..."
24854,"""BLUE PANET II - TURNING VIEWRS INTO CHAMPIONS...","TOWARDS A JOINT INITIATIVE BY MCS, GREENPEACE ...",6000.0,2017,360G-Ellerman-17137,360Giving,"[-0.00074165524,0.02975384,-0.030915955,-0.024...","[-0.008088995,-0.021558955,-0.011668065,-0.013...","[-0.018680874,-0.026616337,-0.002166937,-0.016...",263207,45242,JOHN ELLERMAN FOUNDATION,1004005,44054,MARINE CONSERVATION SOCIETY,THE MARINE CONSERVATION SOCIETY FIGHTS FOR THE...,"3. THE OBJECTS OF THE COMPANY (THE ""OBJECTS"") ...","[-0.015782591,-0.03111454,-0.028229715,-0.0057...","[-0.016880646,-0.021028548,-0.0282325,0.000103...","[-0.014126928,0.028823355,-0.020226076,-0.0331...","[-0.042463362,0.029150238,-0.019922532,-0.0366...",True,"[Scotland, Europe, Turks And Caicos Islands, N...","[Education/training, Arts/culture/heritage/sci...",[The General Public/mankind]
25044,"""BODY VESSEL CLAY – WOMEN RETHINKING CERAMICS""...","""BODY VESSEL CLAY – WOMEN RETHINKING CERAMICS""...",30000.0,2021,360G-Ellerman-18459,360Giving,"[0.02008698,-0.0538104,-0.020445034,0.02325399...","[0.02008698,-0.0538104,-0.020445034,0.02325399...","[0.006766043,-0.025433596,-0.035274163,0.04629...",263207,45468,JOHN ELLERMAN FOUNDATI

In [10]:
#create checkpoint - save df to pickle
# grants_df.to_pickle(checkpoint_folder / "grants_df.pkl")
# print("Saved grants_df to checkpoint")

Saved grants_df to checkpoint


## The Pairs Dataframe

In [ ]:
#create checkpoint - save df to pickle
# pairs_df.to_pickle(checkpoint_folder / "pairs_df.pkl")
# print("Saved pairs_df to checkpoint")

---

# Retrieving Data from Checkpoints

In [4]:
#get checkpoint folder
checkpoint_folder = Path("./10.1_checkpoints/")

#get checkpoint
funders_df = pd.read_pickle(checkpoint_folder / "funders_df.pkl")
grants_df = pd.read_pickle(checkpoint_folder / "grants_df.pkl")
pairs_df = pd.read_pickle(checkpoint_folder / "pairs_df.pkl")

----

In [ ]:
#simulate user's input
user_num = grants_df.loc[0, "recipient_id"]
user_name = grants_df.loc[0, "recipient_name"]
user_activities = grants_df.loc[0, "recipient_activities"]
user_objectives = grants_df.loc[0, "recipient_objectives"]
user_areas = grants_df.loc[0, "recipient_areas"]
user_beneficiaries = grants_df.loc[0, "recipient_beneficiaries"]
user_causes = grants_df.loc[0, "recipient_causes"]

In [ ]:
# step 1 - does a relationship exist
